In [13]:
import pandas as pd
import data_analysis as helper

In [14]:
leading_digits_us = helper.csv_to_dataframe('data/2020-elections-data.csv', 1, 'votes')

ValueError: invalid literal for int() with base 10: ' '

In [6]:
print(helper.data_to_percentage(leading_digits_us, 1))

AttributeError: 'list' object has no attribute 'value_counts'